<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Personality%20Enabled%20MF/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Preprocessing of the reviews from the json files and store in csv files
# input file : json files from amazon 
# Tripadvisor dataset need to be preprocessed as well

In [ ]:
# importig packages
import numpy as np
import scipy.sparse as sp
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
import scipy.spatial as spt
import statistics
import math
from scipy import spatial
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
# importing packages
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.MENTION,p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from textblob import Word
import re
punctuation = re.compile(r'[-.?&!,:;()|0-9]')

In [ ]:
input_file =  r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Magazine_Subscriptions.json'

In [ ]:
# preprocessing 

with open(input_file) as f:
    lines = f.read().splitlines()  # split into different lines
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']
df_inter['json_element'].apply(json.loads)
az = pd.json_normalize(df_inter['json_element'].apply(json.loads))

arr1 = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})
x = pd.DataFrame(arr1.userId.value_counts()).reset_index()
y = x[x['userId']>=5]
arr2 = pd.DataFrame(y)
arr2 = arr2.rename(columns={"userId":"count","index":"userId"})
df = pd.merge(arr1, arr2, on='userId', how='inner')
df = df[df['rating']<6]
df = df[df['rating']>0]
x = pd.DataFrame(df.itemId.value_counts()).reset_index()
y = x[x['itemId']>=5]
arr3 = pd.DataFrame(y)
arr3 = arr3.rename(columns={"itemId":"count","index":"itemId"})
df2 = pd.merge(df, arr3, on='itemId', how='inner')
az = df2
tgt = az

tgt['processed_text'] = ""
tgt['reviewText'].fillna(" ")
array_text = []
for i in range(len(tgt)):
  x = tgt['reviewText'][i]
  word_tokens = word_tokenize(x) if type(x) != float else  " "
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  lemma_words = []
  for words in filtered_sentence:
      word = Word(words).lemmatize()
      lemma_words.append(word)
  punc_words = []
  for words in lemma_words:
      word =  punctuation.sub("", words)
      if len(word) > 0:
        punc_words.append(word.lower())
  line = ""
  for i in punc_words:
    line = line + " " + i.lower()
  array_text.append(line)

tgt['processed_text'] = array_text
az = tgt[['rating', 'userId', 'itemId', 'reviewText', 'processed_text']]
az = az.drop_duplicates(subset=None, keep='first')

In [ ]:
az.to_csv(r'/content/drive/MyDrive/Per_CD_RS/Amazon_magazine_filtered.csv')